<a href="https://colab.research.google.com/github/Wairice/-2.-GIT/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import networkx as nx  # Імпорт бібліотеки для роботи з графами
import random  # Імпорт бібліотеки для генерації випадкових чисел
import matplotlib.pyplot as plt  # Імпорт бібліотеки для візуалізації графів

# Клас, що представляє граф та його властивості та методи
class Graph:
    def __init__(self, size):
        self.size = size  # Розмір графа
        self.G = nx.Graph()  # Створення пустого графа засобами бібліотеки networkx

        # Додавання вершин та ребер до графа
        for i in range(1, size + 1):
            for j in range(1, size + 1):
                self.G.add_node((i, j))
                if i < size:
                    self.G.add_edge((i, j), (i + 1, j), weight=3, edge_size=2)
                if j < size:
                    self.G.add_edge((i, j), (i, j + 1), weight=3, edge_size=2)


    # Метод для видалення ребер з графа та візуалізації його після модифікації
    def modify_and_draw(self, edges_to_remove):
        max_remove_edges = len(self.G.edges) - self.size * self.size + 1

        # Перевірка, чи не видаляється занадто багато ребер
        while edges_to_remove > max_remove_edges:
            print("Error: Too many edges to remove.")
            return

        attempt_count = 0
        while True:
            G_copy = self.G.copy()
            count = edges_to_remove

            # Випадковим чином видаляємо ребра та перевіряємо, чи граф залишається зв'язним
            while count > 0:
                edges = list(G_copy.edges())
                if edges:
                    edge_to_remove = random.choice(edges)
                    G_copy.remove_edge(*edge_to_remove)
                    if nx.is_connected(G_copy):
                        count -= 1
                    else:
                        G_copy.add_edge(*edge_to_remove)

            # Відображаємо модифікований граф та повертаємо його ребра
            pos = {(i, j): (j, -i) for i, j in G_copy.nodes()}
            nx.draw(G_copy, pos, with_labels=True, font_weight='bold', node_size=400, node_color='yellow')
            plt.show()
            return G_copy.edges

# Клас, що представляє агента для руху по графу та його властивості та методи
class GraphAgent:
    def __init__(self, graph_edges, current_position, end_position):
        self.G = nx.Graph()  # Створення пустого графа для агента
        self.edges = graph_edges  # Ребра графа
        self.current_position = current_position  # Поточна позиція агента
        self.end_position = end_position  # Кінцева позиція агента
        self.visited_nodes = set()  # Множина відвіданих вершин
        self.visited_nodes_return = set()  # Множина вершин, які вже були відвідані та повернулись назад
        self.roads_traveled = set()  # Множина пройдених ребер
        self.previous_position = None  # Попередня позиція агента

    # Метод для відображення графа з позначенням кольорами різних перехресть
    def print_edges(self, current_color='yellow'):
        G_copy = self.G.copy()
        G_copy.add_edges_from(self.edges)

        pos = {node: (node[1], -node[0]) for node in G_copy.nodes()}
        node_colors = [
            'pink' if node == self.end_position != self.current_position else
            ('purple' if node == self.current_position == self.end_position else
             current_color if node == self.current_position else
             ('lime' if node in self.visited_nodes else 'yellow'))
            for node in G_copy.nodes()
        ]

        nx.draw(G_copy, pos, with_labels=True, font_weight='bold', node_size=400, node_color=node_colors)
        plt.show()
        print("\n")

    # Метод для вибору доступних вершин при русі агента
    def choose_nodes(self):
        neighbors = [edge[1] for edge in self.edges if edge[0] == self.current_position] + \
                    [edge[0] for edge in self.edges if edge[1] == self.current_position]
        return [node for node in neighbors if node not in self.visited_nodes]

    # Метод для вибору вершин при поверненні агента
    def choose_nodes_return(self):
        neighbors = [edge[1] for edge in self.edges if edge[0] == self.current_position] + \
                    [edge[0] for edge in self.edges if edge[1] == self.current_position]
        return [node for node in neighbors if node in self.visited_nodes and node not in self.visited_nodes_return]

    # Метод для виконання кроку руху агента
    def move(self):
        while True:
            neighbors = self.choose_nodes()  # Вибір доступних для руху вершин навколо поточної позиції агента
            self.visited_nodes.add(self.current_position)  # Додавання поточної вершини до множини відвіданих

            while not neighbors:  # Поки немає доступних вершин для руху
                return_neighbors = self.choose_nodes_return()  # Вибір вершин для повернення агента

                new_position = random.choice(return_neighbors)  # Вибір нової позиції з вершин для повернення

                # Оновлення даних про відвідані вершини та ребра, відображення графа
                self.visited_nodes_return.add(self.current_position)
                self.roads_traveled.add((self.current_position, new_position))
                self.previous_position = self.current_position
                self.print_edges(current_color='blue')  # Позначення поточного місця розташування агента при поверненні
                self.current_position = new_position  # Оновлення поточної позиції агента
                neighbors = self.choose_nodes()  # Перевірка знову доступних вершин для руху

            # -----------------------

            new_position = random.choice([node for node in neighbors if node != self.previous_position])  # Вибір нової позиції для руху

            road = (self.current_position, new_position)  # Оголошення ребра для додавання в множину пройдених ребер
            self.roads_traveled.add(road)  # Додавання пройденого ребра до множини пройдених ребер
            self.previous_position = self.current_position  # Оновлення попередньої позиції агента
            self.print_edges(current_color='red')  # Позначення поточного місця розташування агента при русі
            self.current_position = new_position  # Оновлення поточної позиції агента

            # Перевірка досягнення кінцевої позиції агента
            if self.current_position == self.end_position:
                print("Goal reached!")  # Вивід повідомлення про досягнення мети
                print("All visited nodes: ", self.visited_nodes)  # Вивід усіх відвіданих вершин
                print("All visited edges: ", self.roads_traveled, "\n\nPath:")  # Вивід усіх відвіданих ребер

                G_copy = self.G.copy()
                G_copy.add_edges_from(self.edges)
                G_copy.clear_edges()

                # Візуалізація шляху та графа після досягнення мети
                for road in self.edges:
                    plt.plot([road[0][1], road[1][1]], [-road[0][0], -road[1][0]], color='grey', linestyle='dashed')

                for path in self.roads_traveled:
                    plt.plot([path[0][1], path[1][1]], [-path[0][0], -path[1][0]], color='black')

                pos = {node: (node[1], -node[0]) for node in G_copy.nodes()}
                node_colors = [
                    ('purple' if node == (size, size) else 'red' if node == (1, 1) else 'whitesmoke' if node not in self.visited_nodes else 'lime')
                    for node in G_copy.nodes()]
                nx.draw(G_copy, pos, with_labels=True, font_weight='bold', node_size=400, node_color=node_colors)

                plt.show()  # Візуалізація оновленого графа після досягнення мети

                return True  # Повернення значення True, щоб завершити роботу методу



# Основний цикл програми
while True:
    size = int(input("Enter the size of the graph: "))  # Введення розміру графа
    max_edges_to_remove = (size - 1) * size * 2 - size * size + 1

    if size > 0:
        edges_to_remove = int(input("Enter the number of edges to remove (max {}): ".format(max_edges_to_remove)))

        if edges_to_remove > 0 and edges_to_remove <= max_edges_to_remove:
            graph_instance = Graph(size)  # Створення графа
            graph_edges = graph_instance.modify_and_draw(edges_to_remove)  # Модифікація та відображення графа

            new_graph_instance = GraphAgent(graph_edges, (1, 1), (size, size))  # Створення агента
            new_graph_instance.move()  # Виклик методу для руху агента
            break
        else:
            print("Error: The number of edges to remove should be greater than 0 and at most", max_edges_to_remove)
    else:
        print("Error: The size cannot be less than or equal to zero. Please enter a valid size.")
